# Testing QA4MBES functionality 2: grid coverage testing


In [1]:
from shapely.geometry import shape
import json

In [2]:
## awkward hack to import local modules in Jupyter
import sys
sys.executable
sys.path.append('/mnt/bigdata/frontierSI/qa4mbes-data-pipeline/qa4mbes')

In [3]:
import testcoverage
import getpointcoverage

## 1. Check geotiff grid coverage of a planned area

Does my geotiff raster cover any of the planned survey region? if so how much? if not roughly how far away were we?

### The positive case - we surveyed some of the planned region

In [4]:
%%time

coveragestats = testcoverage.testcoverage('/mnt/bigdata/frontierSI/sampledata/raster/grid1_1m-crs.tiff', '../tests/geotiff-test-grid1.shp')

CPU times: user 34.2 s, sys: 561 ms, total: 34.8 s
Wall time: 34.7 s


In [5]:
coveragestats

{'teststart': '2019-02-27T13:30:26.496565',
 'teststop': '2019-02-27T13:31:01.244891',
 'plannedcoverage': '../tests/geotiff-test-grid1.shp',
 'testswath': '/mnt/bigdata/frontierSI/sampledata/raster/grid1_1m-crs.tiff',
 'percentcovered': None,
 'areacovered': None,
 'centroiddistance': 0.0,
 'minimumdistance': 1.7976931348623157e+308,
 'intersection': None}

### The null case - we didn't survey any of the planned region

In [6]:
nullstats = testcoverage.testcoverage('../tests/4819-100000lines.xyz', '../tests/nulltest.shp')

In [7]:
nullstats

{'teststart': '2019-02-27T13:31:01.280988',
 'teststop': '2019-02-27T13:31:01.969665',
 'plannedcoverage': '../tests/nulltest.shp',
 'testswath': '../tests/4819-100000lines.xyz',
 'percentcovered': None,
 'areacovered': None,
 'centroiddistance': 861.5700123271158,
 'minimumdistance': 183.13389388218107,
 'intersection': None}

## 2. Check BAG coverage of a planned area

Does my bathymetry attributed grid cover any of the planned survey region? if so how much? if not roughly how far away were we?

In [8]:
# handling BAG
import h5py
from io import BytesIO
from lxml import etree


In [9]:
bagfile = h5py.File("../../bag-samples/grid1_ellipsoid_1m.bag")

In [10]:
for item in bagfile["BAG_root"].items():
    print(item)

('elevation', <HDF5 dataset "elevation": shape (6551, 6501), type "<f4">)
('metadata', <HDF5 dataset "metadata": shape (47164,), type "|S1">)
('tracking_list', <HDF5 dataset "tracking_list": shape (0,), type "|V20">)
('uncertainty', <HDF5 dataset "uncertainty": shape (6551, 6501), type "<f4">)


In [11]:
root = bagfile['BAG_root']

In [12]:
metadata = root["metadata"]

In [13]:
metadata.value

array([b'<', b'?', b'x', ..., b'>', b'\n', b''], dtype='|S1')

In [14]:
buffer = BytesIO(metadatanode.value)
metadatatree = etree.parse(buffer)
mroot = metadatatree.getroot()

NameError: name 'metadatanode' is not defined

In [ ]:
#print(etree.tostring(root, pretty_print=True).decode('ascii'))

In [ ]:
for item in mroot:
    print(item.tag)

In [ ]:
mroot.nsmap

In [ ]:
mroot.get('MD_ReferenceSystem')

In [ ]:
mroot.findtext('MD_ReferenceSystem')

In [ ]:

sri = etree.QName(metadataroot.nsmap['gmd'], 'spatialRepresentationInfo').text
mdr = etree.QName(metadataroot.nsmap['gmd'], 'MD_ReferenceSystem').text
ref = etree.QName(metadataroot.nsmap['gmd'], 'referenceSystemIdentifier').text
code = etree.QName(metadataroot.nsmap['gmd'], 'code').text
crs= etree.QName(metadataroot.nsmap['gco'], 'CharacterString').text



In [ ]:
resolution = (
    root
    .find('.//{}'.format(sri))
    .find('.//{}'.format(mdr))
    .find('.//{}'.format(ref))
    .find('.//{}'.format(code))
    .find('.//{}'.format(crs))
)
print(resolution.text, resolution.get('crs'))



In [ ]:
root.nsmap

In [ ]:
sri = etree.QName(root.nsmap['gmd'], 'spatialRepresentationInfo').text
adp = etree.QName(root.nsmap['gmd'], 'axisDimensionProperties').text
dim = etree.QName(root.nsmap['gmd'], 'MD_Dimension').text
res = etree.QName(root.nsmap['gmd'], 'resolution').text
res_meas = etree.QName(root.nsmap['gco'], 'Measure').text

srs = etree.QName(root.nsmap['gco'], 'CharacterString').text

In [ ]:
resolution = root.find('.//{}'.format(res_meas))
print(resolution.text, resolution.get('uom'))

In [ ]:
root.nsmap['gmd'][1]

In [ ]:
root.find('.//{}'.format(srs)).text

In [ ]:
south_bound_lat = etree.QName(metadataroot.nsmap['gmd'], 'southBoundLatitude').text

In [ ]:
metadataroot.find('.//{}'.format(south_bound_lat))

In [ ]:
metadataroot.attrib

### Interpreting results

`testcoverage` returns a python dictionary containing:

- time of test start
- time of test stop
- path to the 'planned' coverage
- path to the survey swathe being tested
- % of planned coverage overlapped by the swath
- area (in metres) of the planned coverage overlapped by the swath
- distance (in metres) between the centroids of the planned coverage and the swath coverage
- minimum distance (in metres) between the planned and survey coverages. This is a rough guide mainly used to see how far away the survey was in the case of no overlap with the planned region
- a GeoJSON polygon describing the intersection of planned coverage and survey coverage

Optionally, it could contain GeoJSON polygons describing the actual coverages used in the test.

**question:** should swath density metrics be included here?